# Scoring Validation

スコアリングエンジンの検証ノートブック

- スコア分布の確認
- 特徴量とスコアの相関
- トップ銘柄の分析
- 時系列でのスコア変動


In [7]:
# 初期設定とインポート
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# プロジェクトルートを自動算出
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "research":
    PROJECT_ROOT = PROJECT_ROOT.parent

SCORING_DIR = PROJECT_ROOT / "data" / "intermediate" / "scoring"
UNIVERSE_DIR = PROJECT_ROOT / "data" / "intermediate" / "universe"

# スタイル設定
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 10

print("プロジェクト:", PROJECT_ROOT)
print("スコアリングDIR:", SCORING_DIR)


プロジェクト: C:\Users\sohta\equity01
スコアリングDIR: C:\Users\sohta\equity01\data\intermediate\scoring


## 1. スコアデータ読み込み


In [8]:
# 最新のスコアを読み込み
latest_scores = SCORING_DIR / "latest_scores.parquet"
if latest_scores.exists():
    df_scores = pd.read_parquet(latest_scores)
    print(f"スコア銘柄数: {len(df_scores)}")
    print(f"\n{df_scores.head()}")
    print(f"\nカラム: {df_scores.columns.tolist()}")
else:
    print("スコアファイルが見つかりません")
    print(f"確認先: {SCORING_DIR}")


スコア銘柄数: 27

        score_raw  score_penalized     sigma    weight
ticker                                                
9434.T   1.488035         1.188035  0.009578  0.121456
3382.T   0.969185         0.969185  0.013172  0.121456
9432.T   0.937482         0.937482  0.011341  0.121456
9433.T   0.536722         0.536722  0.012577  0.121456
6902.T   0.500975         0.500975  0.019706  0.121456

カラム: ['score_raw', 'score_penalized', 'sigma', 'weight']


## 2. スコア分布の確認


In [ ]:
if 'df_scores' in locals() and len(df_scores) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # スコア分布（score_penalizedを使用）
    score_col = "score_penalized" if "score_penalized" in df_scores.columns else "score_raw"
    df_scores[score_col].hist(bins=30, ax=axes[0])
    axes[0].set_title("スコア分布（Penalized Score）")
    axes[0].set_xlabel("Score")
    axes[0].set_ylabel("銘柄数")
    
    # スコア統計
    stats = df_scores[score_col].describe()
    print("スコア統計:")
    print(stats)
    
    # トップ20銘柄
    top20 = df_scores.nlargest(20, score_col).reset_index()
    top20.plot.bar(x="ticker", y=score_col, ax=axes[1])
    axes[1].set_title("Top 20 スコア")
    axes[1].set_xlabel("Ticker")
    axes[1].set_ylabel("Score")
    axes[1].tick_params(axis="x", rotation=45)
    
    plt.tight_layout()
    plt.show()


## 3. 特徴量とスコアの相関


In [ ]:
if 'df_scores' in locals() and len(df_scores) > 0:
    # スコアカラムを選択
    score_col = "score_penalized" if "score_penalized" in df_scores.columns else "score_raw"
    
    # 利用可能な数値カラムで相関を計算
    numeric_cols = df_scores.select_dtypes(include=[np.number]).columns.tolist()
    
    if len(numeric_cols) > 1 and score_col in numeric_cols:
        # 相関行列
        corr_data = df_scores[numeric_cols].corr()
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr_data, annot=True, fmt=".2f", cmap="coolwarm", center=0, 
                    square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
        plt.title("スコアと特徴量の相関行列")
        plt.tight_layout()
        plt.show()
        
        # スコアとの相関
        if score_col in corr_data.columns:
            score_corr = corr_data[score_col].drop(score_col)
            print(f"\n{score_col}との相関:")
            print(score_corr.sort_values(ascending=False))
    else:
        print("相関計算に十分な数値カラムがありません")


## 4. トップ銘柄の詳細分析


In [ ]:
if 'df_scores' in locals() and len(df_scores) > 0:
    score_col = "score_penalized" if "score_penalized" in df_scores.columns else "score_raw"
    top10 = df_scores.nlargest(10, score_col).reset_index()
    
    print("Top 10 銘柄の詳細:")
    display_cols = ["ticker", score_col, "score_raw", "sigma", "weight"]
    available_cols = [col for col in display_cols if col in top10.columns]
    
    print(top10[available_cols].to_string(index=False))


## 5. ユニバースとの比較


In [ ]:
# ユニバースを読み込み
latest_universe = UNIVERSE_DIR / "latest_universe.parquet"
if latest_universe.exists() and 'df_scores' in locals():
    df_universe = pd.read_parquet(latest_universe)
    
    # スコアカラムを選択
    score_col = "score_penalized" if "score_penalized" in df_scores.columns else "score_raw"
    
    # マージ（tickerをインデックスから列に変換）
    df_scores_reset = df_scores.reset_index()
    merged = df_universe.merge(
        df_scores_reset[["ticker", score_col, "weight", "sigma"]], 
        on="ticker", 
        how="inner"
    )
    
    print(f"ユニバース銘柄数: {len(df_universe)}")
    print(f"スコア付き銘柄数: {len(merged)}")
    
    # 流動性ランクとスコアの関係
    if "liquidity_rank" in merged.columns:
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # 散布図
        axes[0].scatter(merged["liquidity_rank"], merged[score_col], alpha=0.6, s=50)
        axes[0].set_xlabel("Liquidity Rank")
        axes[0].set_ylabel("Score")
        axes[0].set_title("流動性ランクとスコアの関係")
        axes[0].grid(True, alpha=0.3)
        
        # ウェイトとスコアの関係
        axes[1].scatter(merged[score_col], merged["weight"], alpha=0.6, s=50)
        axes[1].set_xlabel("Score")
        axes[1].set_ylabel("Weight")
        axes[1].set_title("スコアとウェイトの関係")
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # 統計情報
        print(f"\n流動性ランクと{score_col}の相関:")
        print(merged[["liquidity_rank", score_col]].corr().iloc[0, 1])


KeyError: "None of [Index(['ticker', 'final_score_norm'], dtype='object')] are in the [columns]"